<a href="https://colab.research.google.com/github/tankien76/CS114.K21/blob/master/Sacasm.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Mô tả bài toán
- Input: Các tiêu đề (headlines)
- Output: Tiêu đề châm biếm

# Thu thập dataset
- Thu thập các tiêu đề châm biếm hoặc không từ trang https://www.theonion.com và https://www.huffingtonpost.com

In [1]:
from google.colab import files
uploaded = files.upload()

Saving kaggle.json to kaggle.json


In [2]:
!ls -lha kaggle.json
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

-rw-r--r-- 1 root root 68 Jun 25 02:58 kaggle.json


In [3]:
!kaggle datasets download -d rmisra/news-headlines-dataset-for-sarcasm-detection

  0% 0.00/3.30M [00:00<?, ?B/s]
100% 3.30M/3.30M [00:00<00:00, 107MB/s]


In [4]:
!unzip news-headlines-dataset-for-sarcasm-detection.zip

Archive:  news-headlines-dataset-for-sarcasm-detection.zip
  inflating: Sarcasm_Headlines_Dataset.json  
  inflating: Sarcasm_Headlines_Dataset_v2.json  


# Prepare Data

download stopwords trong thư viện nltk để xử lý các từ thừa trong tiếng anh

In [5]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [6]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

Mount google drive để lấy dữ liệu từ google drive

Import các thư viện sử dụng:
- pandas : đọc dữ liệu file json
- numpy : xử lý dữ liệu text
- nltk : xử lý ngôn ngữ tự nhiên 

In [7]:
import pandas as pd, numpy as np, re
from nltk.stem.porter import PorterStemmer

Thêm thư viện sklearn để thêm các model : LinearSVC, Naive Bayes, Random Forest

In [8]:
from sklearn.model_selection import train_test_split
from sklearn.svm import LinearSVC
from sklearn.model_selection import cross_val_score
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier

Đọc file data từ file Sarcasm_Headlines_Dataset_v2.json

In [9]:
# Loading data from json file
data = pd.read_json("Sarcasm_Headlines_Dataset_v2.json", lines = True)
data.head()

,is_sarcastic,headline,article_link
0,1,thirtysomething scientists unveil doomsday clo...,https://www.theonion.com/thirtysomething-scien...
1,0,dem rep. totally nails why congress is falling...,https://www.huffingtonpost.com/entry/donna-edw...
2,0,eat your veggies: 9 deliciously different recipes,https://www.huffingtonpost.com/entry/eat-your-...
3,1,inclement weather prevents liar from getting t...,https://local.theonion.com/inclement-weather-p...
4,1,mother comes pretty close to using word 'strea...,https://www.theonion.com/mother-comes-pretty-c...


Đưa dữ liệu headline và labels vào biến

In [10]:
# getting features and labels
features = data['headline']
labels = data['is_sarcastic']

Loại bỏ các ký tự đặc biệt, số và các từ không cần thiết trong headline để chuyển hóa thành vector vì .fit không thực hiện khi dữ liệu là string

In [11]:
# Relacing special symbols and digits in headline column
# re stands for Regular Expression
features = features.apply(lambda s : re.sub('[^a-zA-Z]', ' ', s))

In [12]:
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

stop_words = set(stopwords.words('english'))
for i in range (len(features)):
  word_tokens = word_tokenize(features[i])

  filtered_sentence = [w for w in word_tokens if not w in stop_words]

  filtered_sentence = []

  for w in word_tokens:
      if w not in stop_words:
          filtered_sentence.append(w)

Vector hóa dữ liệu:
- Thêm thư viện Tf-idf
- 

In [13]:
from sklearn.feature_extraction.text import TfidfVectorizer
tfIdfVec = TfidfVectorizer(max_features = 5000)
tfIdfVec.fit(features)

TfidfVectorizer(analyzer='word', binary=False, decode_error='strict',
                dtype=<class 'numpy.float64'>, encoding='utf-8',
                input='content', lowercase=True, max_df=1.0, max_features=5000,
                min_df=1, ngram_range=(1, 1), norm='l2', preprocessor=None,
                smooth_idf=True, stop_words=None, strip_accents=None,
                sublinear_tf=False, token_pattern='(?u)\\b\\w\\w+\\b',
                tokenizer=None, use_idf=True, vocabulary=None)

In [14]:
tf_idf_x = tfIdfVec.transform(features)

In [15]:
# getting training and testing data
features_train, features_test, labels_train, labels_test = train_test_split(tf_idf_x.toarray(), labels, test_size = .2, random_state = 0)

In [16]:
# Using linear support vector classifier
lsvc = LinearSVC()
# training the model
lsvc.fit(features_train, labels_train)
# getting the score of train and test data
print("Train accuracy: ",lsvc.score(features_train, labels_train))
print("Test accuracy: ",lsvc.score(features_test, labels_test))

Train accuracy:  0.9183227778991047
Test accuracy:  0.8317610062893082


In [17]:
gnb = GaussianNB()
gnb.fit(features_train, labels_train)
print(gnb.score(features_train, labels_train))  # 78.86
print(gnb.score(features_test, labels_test))    # 73.80

0.8219698624153745
0.7449336128581412


In [18]:
lr = LogisticRegression()
lr.fit(features_train, labels_train)
print(lr.score(features_train, labels_train))   # 88.16
print(lr.score(features_test, labels_test))     # 83.08

0.8853461454466041
0.8322851153039832


In [19]:
rfc = RandomForestClassifier(n_estimators = 10, random_state = 0)
rfc.fit(features_train, labels_train)
print(rfc.score(features_train, labels_train))  # 98.82
print(rfc.score(features_test, labels_test))    # 79.71

0.9882070321030793
0.78319357092942


In [20]:
# from sklearn.ensemble import RandomForestClassifier
# rfc = RandomForestClassifier(n_estimators=500,random_state=42)
# rfc.fit(features_train,labels_train)
# preds = rfc.predict(features_test)
# print('accuracy: ', accuracy_score(labels_test,preds))

In [22]:
# Loading data from json file
data1 = pd.read_json("dataset.json", lines = True, encoding='cp1252' )
data1.head()

,is_sarcastic,headline,article_link
0,0,The Police Officers Who Killed Breonna Taylor ...,https://www.huffingtonpost.co.uk/entry/police-...
1,1,Scientists Discover Mysterious Radio Transmiss...,https://www.theonion.com/scientists-discover-m...
2,0,Historic Anti-Lynching Legislation Blocked In ...,https://www.huffingtonpost.co.uk/entry/histori...
3,1,Democratic Leaders Announce That They’ve Learn...,https://politics.theonion.com/democratic-leade...
4,0,Will A Coronavirus Vaccine Really Be 'Availabl...,https://www.huffingtonpost.co.uk/entry/coronav...


In [23]:
features_val = data1['headline']
labels_val = data1['is_sarcastic']

In [24]:
# Relacing special symbols and digits in headline column
# re stands for Regular Expression
features = features.apply(lambda s : re.sub('[^a-zA-Z]', ' ', s))

In [25]:
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

stop_words = set(stopwords.words('english'))
for i in range (len(features_val)):
  word_tokens = word_tokenize(features_val[i])

  filtered_sentence = [w for w in word_tokens if not w in stop_words]

  filtered_sentence = []

  for w in word_tokens:
      if w not in stop_words:
          filtered_sentence.append(w)

In [26]:
tv1 = tfIdfVec
#features_val = list(features_val)
features_val = tv1.transform(features_val).toarray()

In [27]:
# getting the score of train and test data
print("Val accuracy: ",lsvc.score(features_val, labels_val))

Val accuracy:  0.7665388302972196


In [28]:
print("Val accuracy: ",gnb.score(features_val, labels_val))

Val accuracy:  0.7286673058485139


In [29]:
print("Val accuracy: ",lr.score(features_val, labels_val))

Val accuracy:  0.7722914669223394


In [30]:
print("Val accuracy: ",rfc.score(features_val, labels_val))

Val accuracy:  0.7545541706615532
